In [1]:
%load_ext autoreload
%autoreload 2

In [121]:
from helpers import *
import numpy as np

In [122]:
data = [ 
    'data/social_cocktail.json',
    'data/liquor.json'
]

descriptions, names = [], []

for d in data:
    descriptions += load_data(d, field='description')
    names += load_data(d, field='name')

print('There are {} recipes in the database.'.format(len(descriptions)))

There are 1153 recipes in the database.


In [123]:
descriptions = [clean_string(x) for x in descriptions]
X_desc = flatten_list(descriptions)

names = [clean_string(x) for x in names]
X_names = flatten_list(names)

In [125]:
# Limit the vocabulary size
vocabulary_size=100
X_limit = limit_vocabulary(X_desc, vocabulary_size=vocabulary_size)

# One-hot encode the data
X_limit_transformed, label_encoder, onehot_encoder = encode_categorical(X_limit)
print('There are {} words with a vocabulary size of {}'.format(*X_limit_transformed.shape))

# Transform data into sequences and predictions
seq_length = 10
X, y = sequence_transform(X_limit_transformed, seq_length)
print('The sequence length is {}'.format(seq_length))
print('Observation shape is {}, label shape is {}'.format(X.shape, y.shape))

There are 18607 words with a vocabulary size of 100
The sequence length is 10
Observation shape is (18596, 10, 100), label shape is (18596, 100)


In [126]:
N = X.shape[0]
X_train, y_train = X[:int(N*0.95)], y[:int(N*0.95)]
X_val, y_val = X[int(N*0.95):], y[int(N*0.95):]

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(17666, 10, 100) (17666, 100) (930, 10, 100) (930, 100)


### Create the Neural Network Models

In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [85]:
model = Sequential()
# model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 512)               1255424   
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               51300     
Total params: 1,306,724
Trainable params: 1,306,724
Non-trainable params: 0
_________________________________________________________________
None


In [86]:
# Create the model
batch_size = 32
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=7, batch_size=batch_size)

Train on 17666 samples, validate on 930 samples
Epoch 1/7
17666/17666 [==============================] - 130s - loss: 3.8170 - categorical_accuracy: 0.1390 - val_loss: 3.6740 - val_categorical_accuracy: 0.1645
Epoch 2/7
17666/17666 [==============================] - 174s - loss: 3.1824 - categorical_accuracy: 0.2602 - val_loss: 3.3909 - val_categorical_accuracy: 0.2151
Epoch 3/7
17666/17666 [==============================] - 171s - loss: 2.9204 - categorical_accuracy: 0.3169 - val_loss: 3.2798 - val_categorical_accuracy: 0.2495
Epoch 4/7
17666/17666 [==============================] - 170s - loss: 2.7305 - categorical_accuracy: 0.3508 - val_loss: 3.2522 - val_categorical_accuracy: 0.2624
Epoch 5/7
17666/17666 [==============================] - 185s - loss: 2.6085 - categorical_accuracy: 0.3720 - val_loss: 3.1894 - val_categorical_accuracy: 0.2645
Epoch 6/7
17666/17666 [==============================] - 193s - loss: 2.4953 - categorical_accuracy: 0.3934 - val_loss: 3.1635 - val_categoric

In [115]:
start = np.random.randint(0, len(X)-1)
seed = ''

for i in range(seq_length):
    x = X[start, i].reshape(1, -1)
    seed += reverse_encoding(x, label_encoder, onehot_encoder)[0] + ' '
    
seed = seed.strip()
observation = X[start]
print(seed)

into shot glass float rum on top it over of


In [118]:
result = []

for i in range(20):
    prediction = predict_observation(
        model, np.array([observation]), 32, label_encoder, onehot_encoder, raw_prediction=True
    )
    result.append(prediction)    
    observation = np.vstack((observation[1:, :], prediction))
    
result_np = np.array(result).reshape(20, vocabulary_size)
txt = reverse_encoding(result_np, label_encoder, onehot_encoder)
print(seed, txt)

into shot glass float rum on top it over of ['chilled' 'glass' 'garnish' 'twist' 'ingredients' 'cocktail' 'ice' 'ice'
 'cocktail' 'glass' 'garnish' 'slice' 'serve' 'ingredients' 'cocktail'
 'ice' 'highball' 'ice' 'stir' 'garnish']
